In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import nltk
import numpy as np
import json
from sklearn.model_selection import GridSearchCV
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alici\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
def remove_stopwords(text):
    all_words = text.split(" ")
    clean_text = [i for i in all_words if i not in stopwords and i!=""]
    return " ".join(clean_text)

In [15]:
def remove_html_tags(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [16]:
def lower_texts(text):
    return text.lower()

In [17]:
def clean_text(text):
    clean_text = remove_stopwords(text)
    clean_text = remove_html_tags(clean_text)
    clean_text = lower_texts(clean_text)
    return clean_text

In [18]:
def formatTime(seg):
    min = seg//60
    seg = seg % 60
    return str(min)+"min"+str(round(seg, 2))+"s"

In [22]:
with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\brmoral\gender.json', encoding='utf-8') as f:
    gender_param = json.load(f)
with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\brmoral\age.json', encoding='utf-8') as f:
    age_param = json.load(f)
with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\brmoral\age_fem.json', encoding='utf-8') as f:
    age_fem_param = json.load(f)
with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\brmoral\age_masc.json', encoding='utf-8') as f:
    age_masc_param = json.load(f)

In [23]:
age = []
age_fem = []
age_masc = []
age_sep = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()
    y_train_gender = df_train['GenderClass']
    y_test_gender = df_test['GenderClass']

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    logisticRegr_gender = LogisticRegression(penalty='l2', C=5714.28, solver='lbfgs', multi_class='multinomial')
    logisticRegr_gender.fit(tfidf_train_gender, y_train_gender)
    pred_gender=logisticRegr_gender.predict(tfidf_test_gender)

    # f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    # f1_list.append(f1)
    # print(f1)

    y_train_age = df_train['AgeClass']
    y_test_age = df_test['AgeClass']

    tfidfvec_age = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_age = tfidfvec_age.fit_transform(X_train)
    tfidf_test_age = tfidfvec_age.transform(X_test)

    logisticRegr_age = LogisticRegression(penalty=age_param["k_"+str(i+1)]["params"]["penalty"], C=age_param["k_"+str(i+1)]["params"]["C"], solver='liblinear')
    logisticRegr_age.fit(tfidf_train_age, y_train_age)
    pred_age=logisticRegr_age.predict(tfidf_test_age)

    f1 = metrics.f1_score(y_test_age, pred_age, average='macro')
    age.append(f1)

    df_test["PredictGender"] = pred_gender

    df_train_fem = df_train[df_train["GenderClass"]==1]
    df_train_masc = df_train[df_train["GenderClass"]==0]

    df_test_fem = df_test[df_test["PredictGender"]==1]
    df_test_masc = df_test[df_test["PredictGender"]==0]

    X_train_fem = df_train_fem['concat'].apply(clean_text).to_numpy()
    X_test_fem = df_test_fem['concat'].apply(clean_text).to_numpy()
    y_train_fem = df_train_fem['AgeClass']
    y_test_fem = df_test_fem['AgeClass']

    tfidfvec_fem = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_fem = tfidfvec_fem.fit_transform(X_train_fem)
    tfidf_test_fem = tfidfvec_fem.transform(X_test_fem)

    logisticRegr_fem = LogisticRegression(penalty=age_fem_param["k_"+str(i+1)]["params"]["penalty"], C=age_fem_param["k_"+str(i+1)]["params"]["C"], solver='liblinear')
    logisticRegr_fem.fit(tfidf_train_fem, y_train_fem)
    pred_fem=logisticRegr_fem.predict(tfidf_test_fem)

    f1 = metrics.f1_score(y_test_fem, pred_fem, average='macro')
    age_fem.append(f1)

    X_train_masc = df_train_masc['concat'].apply(clean_text).to_numpy()
    X_test_masc = df_test_masc['concat'].apply(clean_text).to_numpy()
    y_train_masc = df_train_masc['AgeClass']
    y_test_masc = df_test_masc['AgeClass']

    tfidfvec_masc = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_masc = tfidfvec_masc.fit_transform(X_train_masc)
    tfidf_test_masc = tfidfvec_masc.transform(X_test_masc)

    logisticRegr_masc = LogisticRegression(penalty=age_masc_param["k_"+str(i+1)]["params"]["penalty"], C=age_masc_param["k_"+str(i+1)]["params"]["C"], solver='liblinear')
    logisticRegr_masc.fit(tfidf_train_masc, y_train_masc)
    pred_masc=logisticRegr_masc.predict(tfidf_test_masc)

    f1 = metrics.f1_score(y_test_masc, pred_masc, average='macro')
    age_masc.append(f1)

    pred_ages_sep = pred_fem.tolist() + pred_masc.tolist()
    y_test = y_test_fem.to_list() +  y_test_masc.to_list()
    age_sep.append(metrics.f1_score(y_test, pred_ages_sep, average='macro'))

print()
print(sum(age)/len(age))
print(sum(age_fem)/len(age_fem))
print(sum(age_masc)/len(age_masc))
print(sum(age_sep)/len(age_sep))


0.442674300254308
0.40884767737708916
0.45306544827102585
0.4632106560286652


In [26]:
age = []
age_fem = []
age_masc = []
age_sep = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
      
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()
    y_train_gender = df_train['GenderClass']
    y_test_gender = df_test['GenderClass']

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    logisticRegr_gender = LogisticRegression(penalty='l2', C=5714.28, solver='lbfgs', multi_class='multinomial')
    logisticRegr_gender.fit(tfidf_train_gender, y_train_gender)
    pred_gender=logisticRegr_gender.predict(tfidf_test_gender)

    # f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    # f1_list.append(f1)
    # print(f1)

    y_train_age = df_train['AgeClass']
    y_test_age = df_test['AgeClass']

    tfidfvec_age = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_age = tfidfvec_age.fit_transform(X_train)
    tfidf_test_age = tfidfvec_age.transform(X_test)

    logisticRegr_age = LogisticRegression(penalty="l2", C=2857.14, solver='liblinear')
    logisticRegr_age.fit(tfidf_train_age, y_train_age)
    pred_age=logisticRegr_age.predict(tfidf_test_age)

    f1 = metrics.f1_score(y_test_age, pred_age, average='macro')
    age.append(f1)

    df_test["PredictGender"] = pred_gender

    df_train_fem = df_train[df_train["GenderClass"]==1]
    df_train_masc = df_train[df_train["GenderClass"]==0]

    df_test_fem = df_test[df_test["PredictGender"]==1]
    df_test_masc = df_test[df_test["PredictGender"]==0]

    X_train_fem = df_train_fem['concat'].apply(clean_text).to_numpy()
    X_test_fem = df_test_fem['concat'].apply(clean_text).to_numpy()
    y_train_fem = df_train_fem['AgeClass']
    y_test_fem = df_test_fem['AgeClass']

    tfidfvec_fem = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_fem = tfidfvec_fem.fit_transform(X_train_fem)
    tfidf_test_fem = tfidfvec_fem.transform(X_test_fem)

    logisticRegr_fem = LogisticRegression(penalty="l2", C=2857.14, solver='liblinear')
    logisticRegr_fem.fit(tfidf_train_fem, y_train_fem)
    pred_fem=logisticRegr_fem.predict(tfidf_test_fem)

    f1 = metrics.f1_score(y_test_fem, pred_fem, average='macro')
    age_fem.append(f1)

    X_train_masc = df_train_masc['concat'].apply(clean_text).to_numpy()
    X_test_masc = df_test_masc['concat'].apply(clean_text).to_numpy()
    y_train_masc = df_train_masc['AgeClass']
    y_test_masc = df_test_masc['AgeClass']

    tfidfvec_masc = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_masc = tfidfvec_masc.fit_transform(X_train_masc)
    tfidf_test_masc = tfidfvec_masc.transform(X_test_masc)

    logisticRegr_masc = LogisticRegression(penalty="l2", C=2857.14, solver='liblinear')
    logisticRegr_masc.fit(tfidf_train_masc, y_train_masc)
    pred_masc=logisticRegr_masc.predict(tfidf_test_masc)

    f1 = metrics.f1_score(y_test_masc, pred_masc, average='macro')
    age_masc.append(f1)

    pred_ages_sep = pred_fem.tolist() + pred_masc.tolist()
    y_test = y_test_fem.to_list() +  y_test_masc.to_list()
    age_sep.append(metrics.f1_score(y_test, pred_ages_sep, average='macro'))

print()
print(sum(age)/len(age))
print(sum(age_fem)/len(age_fem))
print(sum(age_masc)/len(age_masc))
print(sum(age_sep)/len(age_sep))


0.4490891653331991
0.4098507048507048
0.4854827912340146
0.4866794082827693


In [24]:
f1_list = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()
    y_train_gender = df_train['GenderClass']
    y_test_gender = df_test['GenderClass']

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    logisticRegr_gender = LogisticRegression(penalty='l2', C=5714.28, solver='lbfgs', multi_class='multinomial')
    logisticRegr_gender.fit(tfidf_train_gender, y_train_gender)
    pred_gender=logisticRegr_gender.predict(tfidf_test_gender)

    f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    f1_list.append(f1)
    print(f1)

print()
print(sum(f1_list)/len(f1_list))

0.5646341463414635
0.5569498069498069
0.6565656565656566
0.5750000000000001
0.5718518518518518
0.6222222222222222
0.5904581955597543
0.6554054054054055
0.7794117647058824
0.6386639676113361

0.6211163017213379


In [13]:
with open('brmoral/gender.json', encoding='utf-8') as f:
    gender_param = json.load(f)

In [14]:
f1_list = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()
    y_train_gender = df_train['GenderClass']
    y_test_gender = df_test['GenderClass']

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    logisticRegr_gender = LogisticRegression(penalty=gender_param["k_"+str(i+1)]["params"]["penalty"], C=gender_param["k_"+str(i+1)]["params"]["C"], solver='lbfgs', multi_class='multinomial')
    logisticRegr_gender.fit(tfidf_train_gender, y_train_gender)
    pred_gender=logisticRegr_gender.predict(tfidf_test_gender)

    f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    f1_list.append(f1)
    print(f1)

print()
print(sum(f1_list)/len(f1_list))

0.6554054054054054
0.6061776061776061


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.6694444444444444
0.6386395843174303


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.5354251012145749


...

c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.6868209730751063


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.6029411764705883
0.6694444444444444

0.6451710124705651


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


In [23]:
with open('brmoral/age.json', encoding='utf-8') as f:
    age_param = json.load(f)

In [25]:
age = []
age_fem = []
age_masc = []
age_sep = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()

    y_train_age = df_train['AgeClass']
    y_test_age = df_test['AgeClass']

    tfidfvec_age = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_age = tfidfvec_age.fit_transform(X_train)
    tfidf_test_age = tfidfvec_age.transform(X_test)

    logisticRegr_age = LogisticRegression(penalty=age_param["k_"+str(i+1)]["params"]["penalty"], C=age_param["k_"+str(i+1)]["params"]["C"], solver='liblinear')
    logisticRegr_age.fit(tfidf_train_age, y_train_age)
    pred_age=logisticRegr_age.predict(tfidf_test_age)

    f1 = metrics.f1_score(y_test_age, pred_age, average='macro')
    age.append(f1)
    print(f1)

print()
print(sum(age)/len(age))

0.46331983805668014
0.4171066921319702
0.4916560563619387
0.5295238095238096
0.4689976689976689
0.30845688266977617
0.47670083876980424
0.4222222222222222
0.5239167336728312
0.4023569023569024

0.4504257644763604


In [27]:
age = []
age_fem = []
age_masc = []
age_sep = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()

    y_train_age = df_train['AgeClass']
    y_test_age = df_test['AgeClass']

    tfidfvec_age = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_age = tfidfvec_age.fit_transform(X_train)
    tfidf_test_age = tfidfvec_age.transform(X_test)

    logisticRegr_age = LogisticRegression(penalty="l2", C=2857.14, solver='liblinear')
    logisticRegr_age.fit(tfidf_train_age, y_train_age)
    pred_age=logisticRegr_age.predict(tfidf_test_age)

    f1 = metrics.f1_score(y_test_age, pred_age, average='macro')
    age.append(f1)
    print(f1)

print()
print(sum(age)/len(age))

0.4988288288288289
0.4171066921319702
0.48243114909781576
0.4104377104377104
0.4682477765642066
0.4566347469220246
0.49568749459856526
0.4222222222222222
0.4468186134852801
0.39247641904336666

0.4490891653331991


In [25]:
with open('brmoral/age_fem.json', encoding='utf-8') as f:
    age_fem_param = json.load(f)

In [29]:
age = []
age_fem = []
age_masc = []
age_sep = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()
    y_train_gender = df_train['GenderClass']
    y_test_gender = df_test['GenderClass']

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    logisticRegr_gender = LogisticRegression(penalty=gender_param["k_"+str(i+1)]["params"]["penalty"], C=gender_param["k_"+str(i+1)]["params"]["C"], solver='lbfgs', multi_class='multinomial')
    logisticRegr_gender.fit(tfidf_train_gender, y_train_gender)
    pred_gender=logisticRegr_gender.predict(tfidf_test_gender)

    # f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    # f1_list.append(f1)
    # print(f1)

    df_test["PredictGender"] = pred_gender

    df_train_fem = df_train[df_train["GenderClass"]==1]
    df_test_fem = df_test[df_test["PredictGender"]==1]

    X_train_fem = df_train_fem['concat'].apply(clean_text).to_numpy()
    X_test_fem = df_test_fem['concat'].apply(clean_text).to_numpy()
    y_train_fem = df_train_fem['AgeClass']
    y_test_fem = df_test_fem['AgeClass']

    tfidfvec_fem = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_fem = tfidfvec_fem.fit_transform(X_train_fem)
    tfidf_test_fem = tfidfvec_fem.transform(X_test_fem)

    logisticRegr_fem = LogisticRegression(penalty=age_fem_param["k_"+str(i+1)]["params"]["penalty"], C=age_fem_param["k_"+str(i+1)]["params"]["C"], solver='liblinear')
    logisticRegr_fem.fit(tfidf_train_fem, y_train_fem)
    pred_fem=logisticRegr_fem.predict(tfidf_test_fem)

    f1 = metrics.f1_score(y_test_fem, pred_fem, average='macro')
    age_fem.append(f1)
    print(f1)
    
print()
print(sum(age_fem)/len(age_fem))

0.3238095238095238
0.4262626262626263


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.4481481481481482
0.24444444444444446


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.1904761904761905


...

0.3571428571428572


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.38095238095238093


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.367003367003367

0.384935064935065


In [27]:
age = []
age_fem = []
age_masc = []
age_sep = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()
    y_train_gender = df_train['GenderClass']
    y_test_gender = df_test['GenderClass']

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    logisticRegr_gender = LogisticRegression(penalty="l2", C=5714.28, solver='lbfgs', multi_class='multinomial')
    logisticRegr_gender.fit(tfidf_train_gender, y_train_gender)
    pred_gender=logisticRegr_gender.predict(tfidf_test_gender)

    # f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    # f1_list.append(f1)
    # print(f1)

    df_test["PredictGender"] = pred_gender

    df_train_fem = df_train[df_train["GenderClass"]==1]
    df_test_fem = df_test[df_test["PredictGender"]==1]

    X_train_fem = df_train_fem['concat'].apply(clean_text).to_numpy()
    X_test_fem = df_test_fem['concat'].apply(clean_text).to_numpy()
    y_train_fem = df_train_fem['AgeClass']
    y_test_fem = df_test_fem['AgeClass']

    tfidfvec_fem = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_fem = tfidfvec_fem.fit_transform(X_train_fem)
    tfidf_test_fem = tfidfvec_fem.transform(X_test_fem)

    logisticRegr_fem = LogisticRegression(penalty="l2", C=2857.14, solver='liblinear')
    logisticRegr_fem.fit(tfidf_train_fem, y_train_fem)
    pred_fem=logisticRegr_fem.predict(tfidf_test_fem)

    f1 = metrics.f1_score(y_test_fem, pred_fem, average='macro')
    age_fem.append(f1)
    print(f1)
    
print()
print(sum(age_fem)/len(age_fem))

0.3238095238095238
0.41269841269841273
0.5151515151515151
0.38888888888888884
0.2222222222222222
0.7074074074074074
0.3333333333333333
0.5301587301587302
0.48333333333333334
0.28571428571428575

0.4202717652717653


In [39]:
with open('brmoral/age_masc.json', encoding='utf-8') as f:
    age_masc_param = json.load(f)

In [40]:
age = []
age_fem = []
age_masc = []
age_sep = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()
    y_train_gender = df_train['GenderClass']
    y_test_gender = df_test['GenderClass']

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    logisticRegr_gender = LogisticRegression(penalty=gender_param["k_"+str(i+1)]["params"]["penalty"], C=gender_param["k_"+str(i+1)]["params"]["C"], solver='lbfgs', multi_class='multinomial')
    logisticRegr_gender.fit(tfidf_train_gender, y_train_gender)
    pred_gender=logisticRegr_gender.predict(tfidf_test_gender)

    # f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    # f1_list.append(f1)
    # print(f1)

    df_test["PredictGender"] = pred_gender

    df_train_masc = df_train[df_train["GenderClass"]==0]
    df_test_masc = df_test[df_test["PredictGender"]==0]

    X_train_masc = df_train_masc['concat'].apply(clean_text).to_numpy()
    X_test_masc = df_test_masc['concat'].apply(clean_text).to_numpy()
    y_train_masc = df_train_masc['AgeClass']
    y_test_masc = df_test_masc['AgeClass']

    tfidfvec_masc = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_masc = tfidfvec_masc.fit_transform(X_train_masc)
    tfidf_test_masc = tfidfvec_masc.transform(X_test_masc)

    logisticRegr_masc = LogisticRegression(penalty=age_masc_param["k_"+str(i+1)]["params"]["penalty"], C=age_masc_param["k_"+str(i+1)]["params"]["C"], solver='liblinear')
    logisticRegr_masc.fit(tfidf_train_masc, y_train_masc)
    pred_masc=logisticRegr_masc.predict(tfidf_test_masc)

    f1 = metrics.f1_score(y_test_masc, pred_masc, average='macro')
    age_masc.append(f1)
    print(f1)


print()
print(sum(age_masc)/len(age_masc))

0.3855500821018063
0.4962406015037594


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.4591269841269842
0.5167534220165798


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.32861952861952864


...

0.49901477832512314


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.5409713743047077


c:\Users\alici\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1116: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


0.3819195539625648

0.44915775222272447


In [32]:
age = []
age_fem = []
age_masc = []
age_sep = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()
    y_train_gender = df_train['GenderClass']
    y_test_gender = df_test['GenderClass']

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    logisticRegr_gender = LogisticRegression(penalty="l2", C=5714.28, solver='lbfgs', multi_class='multinomial')
    logisticRegr_gender.fit(tfidf_train_gender, y_train_gender)
    pred_gender=logisticRegr_gender.predict(tfidf_test_gender)

    # f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    # f1_list.append(f1)
    # print(f1)

    df_test["PredictGender"] = pred_gender

    df_train_masc = df_train[df_train["GenderClass"]==0]
    df_test_masc = df_test[df_test["PredictGender"]==0]

    X_train_masc = df_train_masc['concat'].apply(clean_text).to_numpy()
    X_test_masc = df_test_masc['concat'].apply(clean_text).to_numpy()
    y_train_masc = df_train_masc['AgeClass']
    y_test_masc = df_test_masc['AgeClass']

    tfidfvec_masc = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_masc = tfidfvec_masc.fit_transform(X_train_masc)
    tfidf_test_masc = tfidfvec_masc.transform(X_test_masc)

    logisticRegr_masc = LogisticRegression(penalty="l2", C=2857.14, solver='liblinear')
    logisticRegr_masc.fit(tfidf_train_masc, y_train_masc)
    pred_masc=logisticRegr_masc.predict(tfidf_test_masc)

    f1 = metrics.f1_score(y_test_masc, pred_masc, average='macro')
    age_masc.append(f1)
    print(f1)


print()
print(sum(age_masc)/len(age_masc))

0.5137147335423197
0.5703703703703703
0.4143199233716475
0.5167534220165798
0.4078431372549019
0.3959131545338442
0.5131815044858523
0.5239234449760765
0.5402969790066564
0.3078703703703704

0.4704187039928619
